## Continuous Attribute 나누기

불연속적 명목 데이터에 비해 나눌 수 있는 구간이 많음
1) 전체 데이터를 모두 기준점으로 한다
2) 중위값, 4분위수들을 기준점으로 한다 (binning)
3) **Y-class값이 바뀌는 수를 기준점으로 한다**

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
pd_data = pd.read_csv('data/vegeterianl_dataset.csv',  delimiter=r"\s+")
pd_data.drop("ID",axis=1)
pd_data

,ID,STREAM,SLOPE,ELEVATION,VEGETATION
0,1,False,steep,3900,chapparal
1,2,True,moderate,300,riparian
2,3,True,steep,1500,riparian
3,4,False,steep,1200,chapparal
4,5,False,flat,4450,conifer
5,6,True,steep,5000,conifer
6,7,True,steep,3000,chapparal


In [3]:
# ELEVATION : continuous value
pd_data.sort_values("ELEVATION")

,ID,STREAM,SLOPE,ELEVATION,VEGETATION
1,2,True,moderate,300,riparian
3,4,False,steep,1200,chapparal
2,3,True,steep,1500,riparian
6,7,True,steep,3000,chapparal
0,1,False,steep,3900,chapparal
4,5,False,flat,4450,conifer
5,6,True,steep,5000,conifer


In [4]:
def get_info(df):
    riparian = df.loc[df["VEGETATION"]=="riparian"]
    chapparal = df.loc[df["VEGETATION"]=="chapparal"]
    conifer = df.loc[df["VEGETATION"]=="conifer"]

    x = np.array([len(riparian)/len(df),len(chapparal)/len(df), 
                  len(conifer)/len(df)])
    y = np.log2(x[x!=0])
    

    info_all = - sum(x[x!=0] * y) # entropy
    return info_all

In [5]:
get_info(pd_data)

1.5566567074628228

In [6]:
def get_attribute_info(df, attribute_name, continuous_value=0):
    get_infos = []
    if continuous_value == 0: # 해당 attribute가 연속적인 값이 아닐 때 (continuous_value = 0일 때)
        attribute_values = pd_data[attribute_name].unique()
        for value in attribute_values:
            split_df = pd_data.loc[pd_data[attribute_name] == value]
            get_infos.append((len(split_df) / len(df)) * get_info(split_df))
    else: # 해당 attribute가 연속적인 값일 때 (continuous_value가 split의 기준값)
        split_df_1 = pd_data.loc[pd_data[attribute_name] >= continuous_value]
        split_df_2 = pd_data.loc[pd_data[attribute_name] < continuous_value]

        get_infos.append((len(split_df_1) / len(df)) * get_info(split_df_1))                     
        get_infos.append((len(split_df_2) / len(df)) * get_info(split_df_2))                     
    return sum(get_infos)

In [8]:
def get_continuos_attribute_info(df, attribute_name):
    sorted_df = df.sort_values(attribute_name)
    prior = 0 # y의 초기 기준값 : 0 (y가 바뀌는 지점을 찾아 split_index에 추가하게 됨)
    split_index = []
    rank_to_index = {}
    index_to_rank = {}
    for rank, index in enumerate(sorted_df.index):
      # rank : 순위 (그냥 0부터 1씩 증가), index : 기존 df에서 부여받은 index (ID 값 아님)
      if prior != sorted_df["VEGETATION"][index]:
        split_index.append(index) # 값이 바뀌는 지점 저장
        prior = sorted_df["VEGETATION"][index] # prior를 바뀐 지점에서의 y 값으로 업데이트
      index_to_rank[index] = rank
      rank_to_index[rank] = index

    split_value = []
    for v in split_index[1:]:  # 0번째는 prior 초기값 0에서 첫 y값으로 바뀐 경우이므로 제외
      current_value = sorted_df[attribute_name][v]
      prior_value_index = rank_to_index[index_to_rank[v] - 1]
      prior_value = sorted_df[attribute_name][prior_value_index]
      
      split_value_average = current_value + prior_value
      split_value.append(split_value_average/2)
    result = {}
    for value in split_value:
        result[value] = (get_attribute_info(df, attribute_name, value))
    return result

In [9]:
get_continuos_attribute_info(pd_data, "ELEVATION")

{750.0: 1.250698214594781,
 1350.0: 1.3728057820624016,
 2250.0: 0.9649839288804954,
 4175.0: 0.6935361388961918}

In [10]:
for key, value in get_continuos_attribute_info(pd_data, "ELEVATION").items():
    print(key, ":", get_info(pd_data) - value)

750.0 : 0.3059584928680419
1350.0 : 0.18385092540042125
2250.0 : 0.5916727785823274
4175.0 : 0.863120568566631


In [11]:
get_info(pd_data) - get_attribute_info(pd_data, "STREAM")

0.30595849286804166

In [12]:
get_info(pd_data) - get_attribute_info(pd_data, "SLOPE")

0.5774062828523452

## Continuous attribute branch 특징

- 명목속성과 달리, 여러번 재사용 가능 (단, 경계값은 달라야 함)
- 연속값과 명목값을 번갈아가면서 split 가능 (성능상 좋고 나쁨은 데이터에 따라 다름)